In [79]:
import requests
import json
from newspaper import Article
from bs4 import BeautifulSoup

In [108]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows; U; MSIE 9.0; WIndows NT 9.0; ko-KR))',
}

req = requests.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=", headers=headers)

html = req.text
soup = BeautifulSoup(html, "lxml")

lst_realtime_src = soup.find('ol', {"class":"lst_realtime_srch _tab_area"})
lis = lst_realtime_src.findAll('li')

articles = {"article":[]}
index = 1

for li in lis:
#   News Url
    keyword = li.find("span", {"class":"tit"})
    news_page_url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=" + keyword.text
    req = requests.get(news_page_url, headers=headers)
    html = req.text
    soup = BeautifulSoup(html, "lxml")
    
    news_list = soup.find("ul", {"class":"type01"}).findAll('li')[0]
    source = news_list.find("span", {"class":"_sp_each_source"})
    link = news_list.find("a")["href"]
    
    req = requests.get(link, headers=headers)
    html = req.text

    a = Article(link, language="ko")
    a.download()
    a.parse()
    
    article = {
        "number" : index,
        "keyword" : keyword,
        "title" : a.title,
        "content" : a.text,
        "source" : source.text,
        "image" : a.top_image
    }
    
    print(a.authors)
    
#     articles["article"].append(article)
    
# print(articles)
# print('hello')
    

[]
[]
['권애리 기자']
[]
['부산 여중생 폭행 가해자 추가 구속', '도망할 염려 있어']
[]
[]
['괴짜 노벨상', '걸을 때 커피 쏟는 이유 규명한 한국인 수상']
[]
[]
